In [2]:
pip install dill

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import dill
import pandas as pd
import os
import re
import json
from collections import Counter
import time

In [4]:
ENVS = {'click-checkboxes-soft':10,
 'click-tab-2-hard':20,
 'social-media':15,
 'email-inbox':30,
 'social-media-some':30,
 'tic-tac-toe':10,
 'use-autocomplete':10,
 'book-flight':30,
 'choose-date':20,
 'search-engine':20}

In [5]:
def remove_nonessentials(dom):
    elements = []
    for e in dom:
        element = e.copy()
        del element['fg_color']
        del element['bg_color']
        del element['flags']
        element['left'] = int(element['left'][0])
        element['top'] = int(element['top'][0])
        element['height'] = int(element['height'][0])
        element['width'] = int(element['width'][0])
        elements.append(element)
    return elements

def remove_nonessentials_sms(dom):
    elements = []
    for e in dom:
        element = e.copy()
        del element['fg_color']
        del element['bg_color']
        element['flags'] = element['flags'].astype(int)
        element['left'] = int(element['left'][0])
        element['top'] = int(element['top'][0])
        element['height'] = int(element['height'][0])
        element['width'] = int(element['width'][0])
        elements.append(element)
    return elements

In [11]:
def create_jsonl(trajectory,env):
    states = trajectory['states']
    doms = [x['dom_elements'] for x in states]
    images = [x['screenshot'] for x in states]
    actions = [x['action'] for x in states]
    goals = [trajectory['utterance']]* (len(states)-1)
    df_ref = pd.DataFrame()
    df_ref['goal'] = goals
    df_ref['dom'] =doms[:-1]
    if env == 'social-media-some':
        df_ref['dom_simplified'] = df_ref['dom'].copy().apply(remove_nonessentials_sms)
    else:
        df_ref['dom_simplified'] = df_ref['dom'].copy().apply(remove_nonessentials)

    df_ref['output'] = actions[1:]
    df_train = pd.DataFrame()
    df_train['input'] = df_ref.apply(lambda x: 'Goal: {} DOM elements: {}'.format(x['goal'], x['dom']).replace(
        ', dtype=float32)','').replace(', dtype=int8)','').replace('array(','').replace("'",""), axis=1)
    df_train['output'] = df_ref['output']
    lst = []
    for row in df_train.itertuples():
        lst.append(json.dumps({"messages":[{"role":"user","content":row[1]},{"role":"model","content":row[2]}]}))
    return lst

def create_test_dataset(env,df_stats):
    jsonl = []
    filenames = df_token_counts[(df_stats.env_name == env) & (df_stats.max_tokens_cn_short<=8192)].filename.values
    for filename in filenames:
        trajectory = dill.load(open(filename,'rb'))
        jsonl += create_jsonl(trajectory,env)
    dataset_name ='dataset_{}.JSONL'.format(env)
    print(dataset_name,len(jsonl))
    with open(dataset_name, 'w') as file:
        for example in jsonl:
            file.write(example + '\n')
    return dataset_name,len(jsonl)

def concatenate_jsonl_files(input_files, output_file):
    count_examples = 0
    with open(output_file, 'w') as outfile:
        for file_path in input_files:
            with open(file_path, 'r') as infile:
                for line in infile:
                    outfile.write(line)
                    count_examples +=1
    print('Total examples: {}', count_examples)


In [12]:
input_files, lengths = [],[]
df_token_counts = dill.load(open('training_stats.pkd','rb'))
for env in ENVS:
    dataset_name,length = create_test_dataset(env, df_token_counts)
    lengths.append(length)
    input_files.append(dataset_name)
df = pd.DataFrame()
df['Environment'] = ENVS.keys()
df['Number of Examples'] = lengths
df.to_csv('dataset_composition1.csv')
concatenate_jsonl_files(input_files, 'dataset_full_test.JSONL' )
df

dataset_click-checkboxes-soft.JSONL 323
dataset_click-tab-2-hard.JSONL 263
dataset_social-media.JSONL 163
dataset_email-inbox.JSONL 280
dataset_social-media-some.JSONL 280
dataset_tic-tac-toe.JSONL 341
dataset_use-autocomplete.JSONL 412
dataset_book-flight.JSONL 1224
dataset_choose-date.JSONL 838
dataset_search-engine.JSONL 456
Total examples: {} 4580


,Environment,Numbr of Examples
0,click-checkboxes-soft,323
1,click-tab-2-hard,263
2,social-media,163
3,email-inbox,280
4,social-media-some,280
5,tic-tac-toe,341
6,use-autocomplete,412
7,book-flight,1224
8,choose-date,838
9,search-engine,456


In [13]:
df = df.rename(columns={'Numbr of Examples':'Number of Examples'})
df.to_csv('dataset_composition1.csv')
df

,Environment,Number of Examples
0,click-checkboxes-soft,323
1,click-tab-2-hard,263
2,social-media,163
3,email-inbox,280
4,social-media-some,280
5,tic-tac-toe,341
6,use-autocomplete,412
7,book-flight,1224
8,choose-date,838
9,search-engine,456


In [16]:
df_token_counts[df_token_counts.max_tokens_cn_short<=8192].groupby('env_name').agg('count').filename

env_name
book-flight              111
choose-date              100
click-checkboxes-soft    100
click-tab-2-hard         100
email-inbox              108
search-engine             91
social-media             105
social-media-some        107
tic-tac-toe              100
use-autocomplete         103
Name: filename, dtype: int64

In [18]:
df = df.set_index('Environment')
df['Number of Trajectories'] = df_token_counts[df_token_counts.max_tokens_cn_short<=8192].groupby('env_name').agg('count').filename

KeyError: "None of ['Environment'] are in the columns"

In [21]:
df=df.reset_index()[['Environment','Number of Trajectories','Number of Examples']]
df

,Environment,Number of Trajectories,Number of Examples
0,click-checkboxes-soft,100,323
1,click-tab-2-hard,100,263
2,social-media,105,163
3,email-inbox,108,280
4,social-media-some,107,280
5,tic-tac-toe,100,341
6,use-autocomplete,103,412
7,book-flight,111,1224
8,choose-date,100,838
9,search-engine,91,456


In [22]:
df.to_csv('dataset_composition1.csv')

In [15]:
trajectory = dill.load(open('trajectories_train/tic-tac-toe/tic-tac-toe036.pkd', 'rb'))
print(trajectory['utterance'])
print(trajectory['reward'])
[x['action'] for x in trajectory['states']]

Playing as 'X', win a game of tic-tac-toe.
0.8200000000000001


['',
 'Click on any empty square - CLICK_ELEMENT 5',
 'Click on a square in a row, column or diagonal where there is an X and an empty space and no O\n        or two Xs and no O - CLICK_ELEMENT 6',
 'Click on a square in a row, column or diagonal where there is an X and an empty space and no O\n        or two Xs and no O - CLICK_ELEMENT 9',
 'Click on a square in a row, column or diagonal where there is an X and an empty space and no O\n        or two Xs and no O - CLICK_ELEMENT 13']